In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import numpy as np
from graphdatascience import GraphDataScience
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import neo4jupyter
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops, to_dense_adj, dense_to_sparse
from torch_geometric.nn import VGAE,GAE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from torch_geometric.utils import to_dense_adj
#neo4jupyter.init_notebook_mode()
import tqdm as notebook_tqdm

In [3]:
import pickle
# Load the list of arrays from the .pkl file
with open('med_node_embeddings_50d.pkl', 'rb') as pkl_file:
    extracted_data = pickle.load(pkl_file)

In [4]:
node_embeddings = extracted_data

In [ ]:
diseases=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR_med\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Disease.csv')
Med=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR_med\\drug_disease.csv')

In [ ]:
diseases_nodes=list(diseases.fname)
medication_nodes=list(medication.fname)

In [ ]:
diseases_embeddings = {key:node_embeddings[key] for key in diseases_nodes if key in node_embeddings}
medication_embeddings = {key:node_embeddings[key] for key in medication_nodes if key in node_embeddings}

In [8]:
HF_ind = ['Chlorthalidone','Mannitol','Acetazolamide','Etacrynic acid','Torasemide','Piretanide','Furosemide','Spironolactone','Triamterene',
          'Bumetanide','Indapamide','Amiloride','Xipamide','Hydroflumethiazide','Methyclothiazide','Fosinopril','Ramipril','Benazepril',
          'Quinapril','Perindopril','Lisinopril','Carvedilol','Bisoprolol','Captopril','Metoprolol','Diltiazem','Atenolol','Candesartan cilexetil',
          'Telmisartan','Irbesartan','Valsartan','Eprosartan','Losartan','Moexipril','Ibopamine','Enalaprilat','Enalapril','Ubidecarenone',
          'Azosemide','Levosimendan','Acetyldigitoxin']

In [9]:
HF_cont = ['Monopotassium phosphate','Polyethylene glycol 300','Magnesium trisilicate','nitrogen','Chlorphentermine','Methenamine',
           'Bisacodyl','Mitomycin','Phenylacetic acid','Dexrabeprazole','Oxandrolone','Fenoprofen','Phenyl salicylate','Anagrelide',
           'Bendroflumethiazide','Hyoscyamine','Tesamorelin','Carteolol','Diazoxide','Sodium chloride','Galactose','Ioxaglic acid',
           'Iron sucrose','Cefpodoxime','Telavancin','Gadodiamide','Ertapenem','Meropenem','Iodixanol','Sorbitol','Ioversol',
           'Iopromide','Daptomycin','Iopamidol','Tigecycline','Mannitol','Rimantadine','Gadobenic acid','Linezolid','Vidarabine',
           'Triamcinolone']

In [12]:
list_all_med = list(set(Med.y_name))

In [ ]:
list_of_pos_neg = HF_ind+HF_cont

In [ ]:
list_uknown_med = [item for item in list_all_med if item not in list_of_pos_neg]

In [ ]:
med_embeddigns_pos = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('heart failure'))) for key in HF_ind if key in medication_embeddings]
med_embeddigns_neg = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('heart failure'))) for key in HF_cont if key in medication_embeddings]
uknown_medication_embeddings = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('heart failure'))) for key in list_uknown_med if key in medication_embeddings]
HF_embeddings = diseases_embeddings.get('heart failure')

In [18]:
med_disease_pos = pd.DataFrame(np.array(med_embeddigns_pos))
med_disease_pos['Label'] = 1
med_disease_neg = pd.DataFrame(np.array(med_embeddigns_neg))
med_disease_neg['Label'] = 0

In [19]:
med_disease_pos_neg = pd.concat([med_disease_pos,med_disease_neg])

In [ ]:
uknown_med_names= [key for key in list_uknown_med if key in medication_embeddings]
uknown_med_embeddigns_df = pd.DataFrame(np.array(uknown_medication_embeddings))

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import random 

random.seed(123456)
np.random.seed(123456)
X = med_disease_pos_neg.iloc[:,0:-1]
y = med_disease_pos_neg.iloc[:,-1]
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=20)

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 5, 10, 50, 100, 200, 300, 500],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Initialize the classifier
random.seed(123)
svm = SVC(probability=True, random_state=42)

# Initialize GridSearchCV
random.seed(123)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print("Best parameters found: ", best_params)

# Evaluate the best estimator
y_pred_svm = best_svm.predict(X_test)
y_pred_proba_svm = best_svm.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_svm))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'C': 50, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.8
ROC AUC: 0.7142857142857143


In [30]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

random.seed(123)
# Define the tuned parameters (example parameters, replace with your actual tuned values)
tuned_params = {
    'C': 50,        # Regularization parameter
    'gamma': 1, # Kernel coefficient
    'kernel': 'rbf' # Kernel type
}

# Initialize the classifier with tuned parameters
svm = SVC(probability=True, random_state=42, **tuned_params)

# Train the classifier
svm.fit(X_train, y_train)

# Predict the test set labels
y_pred_svm = svm.predict(X_test)
y_pred_proba_svm = svm.predict_proba(X_test)[:, 1]

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_svm))


Accuracy: 0.8
ROC AUC: 0.7142857142857143


In [ ]:
svm_my_dict = {}
for i in range(0,uknown_med_embeddigns_df.shape[0]):
    y_pred_svm_p = svm.predict(uknown_med_embeddigns_df.iloc[i:i+1,:])
    y_pred_proba_svm_p = svm.predict_proba(uknown_med_embeddigns_df.iloc[i:i+1,:])[:, 1]
    print(uknown_med_names[i],y_pred_svm_p[0],y_pred_proba_svm_p[0])
    if y_pred_svm_p[0]>0:
        svm_my_dict[uknown_med_names[i]] =y_pred_proba_svm_p[0]

pos_prd_prob_svm = dict(sorted(svm_my_dict.items(), key=lambda item: item[1],reverse=True))